## Project Modules

In [1]:
from project_modules.project_imports import *

from project_modules.classes import Patient
from project_modules.classes import Clinic

from project_modules.rules import call_a_rule

from project_modules.simulation_tools import create_appointments, plot_line_graph
from project_modules.simulation_tools import stablish_attendance, random_patient_sample
from project_modules.simulation_tools import get_margin_errors, check_convergence_mean, confidence_interval, calculate_summary

## Prediction Model

In [2]:
X_file_path = "prediction_model/X.npy"
y_file_path = "prediction_model/y.npy"
column_file_path = "prediction_model/column_names.pkl"

with open(column_file_path, 'rb') as file:
    column_names = pickle.load(file)

X = np.load(X_file_path, allow_pickle=True)
y = np.load(y_file_path, allow_pickle=True)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X)
X.columns = column_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

model_file_path = "prediction_model/Final_model.pkl"
with open(model_file_path, 'rb') as file:
    model = pickle.load(file)

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)
y_pred_proba_1 = y_pred_proba[:,-1]

assert roc_auc_score(y_test, y_pred_proba_1) > 0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Initialization

In [3]:
patients_original = [Patient(i, **row) for i, row in enumerate(X.to_dict(orient='records'))]
patients = patients_original.copy()

# Extracting keys from the first patient object
patient_keys = set(patients[0].__dict__.keys())

# Converting column_names to a set for faster comparison
column_name_set = set(column_names)

# Finding keys that are in patient_keys but not in column_names
missing_keys = patient_keys - column_name_set

verbose = False
if verbose:
    print("Keys in patients[0].__dict__.keys() but not in column_names:")
    print(missing_keys)
    len(missing_keys)

In [4]:
'''
# Si se necesita volver a calcular las probabilidades (~1min 39s)
patients_with_proba = []
for patient in patients:
    patient.predict_proba(model)
    patients_with_proba.append(patient)

# export patients_with_proba to a pkl file
import pickle
with open('patients_with_proba_may5.pkl', 'wb') as f:
    pickle.dump(patients_with_proba, f)
''' 

# Cargando con las probabilidades ya hechas
with open('patients_with_proba.pkl', 'rb') as f:
    patients_with_proba = pickle.load(f)

patients = patients_with_proba.copy()
probas = [patient.proba for patient in patients]

plotting = False
if plotting:
    plt.figure(figsize=(10,5))
    plt.hist(probas, bins=50)
    plt.xlabel('Predicted probability')
    plt.ylabel('Number of patients')
    plt.title('Predicted probability distribution')
    plt.show()

ppv_npv_results_file_path = "prediction_model/ppv_npv_results.xlsx"
ppv_npv_df = pd.read_excel(ppv_npv_results_file_path)

## Simulation

### Parameters

In [5]:
seed = 42
slot_time = 20
num_serves = 1
work_hours = 10
overbooking = 1.0
simulation_days = 7
num_slots_byday = (60//slot_time) * work_hours
available_slots = num_slots_byday* simulation_days 

# extra percentage for appointments
extra_pct = 1.35
sample_size = int(np.ceil(available_slots + (available_slots * extra_pct)))

thresholds_to_evaluate = []
for threshold1 in [x * 0.1 for x in range(1, 10)]:
    for threshold2 in [x * 0.1 for x in range(1, 10)]:
        thresholds_to_evaluate.append((round(threshold1,2), round(threshold2,2)))

sample_protected_pct = 0.135789

In [6]:
rule_name = "ATBEG"
param_overbooking = 4 

In [7]:
# Parameters
rule_name = "ATBEG"
param_overbooking = 3


In [8]:
plot_prefix = f"{rule_name}{param_overbooking}_3high_sample"
num_replicas, iterations_per_schedule = 30, 30
verbose, debug_verbose = False, False

### Process

In [9]:
# crear el workbook de excel
wb = xl.Workbook()
ws = wb.active

border_style = Border(left=Side(style='thin'),
                      right=Side(style='thin'),
                      top=Side(style='thin'),
                      bottom=Side(style='thin'))

# region column widths
default_column_width = 11
ws.column_dimensions["A"].width = 25
ws.column_dimensions["B"].width = default_column_width
ws.column_dimensions["C"].width = default_column_width
ws.column_dimensions["D"].width = default_column_width
ws.column_dimensions["E"].width = default_column_width
ws.column_dimensions["F"].width = default_column_width
ws.column_dimensions["G"].width = 2 * default_column_width
ws.column_dimensions["H"].width = 27
ws.column_dimensions["I"].width = default_column_width
ws.column_dimensions["J"].width = default_column_width
ws.column_dimensions["K"].width = default_column_width
ws.column_dimensions["L"].width = default_column_width
ws.column_dimensions["M"].width = 2 * default_column_width
ws.column_dimensions["N"].width = 2 * default_column_width
ws.column_dimensions["O"].width = 2 * default_column_width
ws.column_dimensions["P"].width = 2 * default_column_width
ws.column_dimensions["Q"].width = 2 * default_column_width
ws.column_dimensions["R"].width = 2 * default_column_width
ws.column_dimensions["S"].width = 2 * default_column_width
ws.column_dimensions["T"].width = 2 * default_column_width
# endregion

# region headers excel

'''
header_names = [
    "Rule",
    "Sample Proportion",
    "",
    "Threshold",
    "",
    "Iterations",
    "Comments",
    "Metric",
    "Mean",
    "Interval",
    "",
    "Margin",
    "p-value non corr",
    "",
    "t non corr",
    "",
    "p-value corr",
    "",
    "t corr",
    ""
]
'''

header_names = [
    "rule",
    "protected_proportion",
    "unprotected_proportion",
    "protected_threshold",
    "unprotected_threshold",
    "iterations",
    "comments",
    "metric",
    "mean",
    "lower_bound",
    "upper_bound",
    "margin",
    "one_sided_pval_non_corr",
    "two_sided_pval_non_corr",
    "one_sided_t_non_corr",
    "two_sided_t_non_corr",
    "one_sided_pval_corr",
    "two_sided_pval_corr",
    "one_sided_t_corr",
    "two_sided_t_corr"
]

for i, header in enumerate(header_names):
    ws.cell(row=1, column=i+1, value=header)
    
    # if second_header_names[i]:  
    #     ws.cell(row=2, column=i+1, value=second_header_names[i])
    #     ws.cell(row=2, column=i+1).fill = PatternFill(start_color="00024A", end_color="00024A", fill_type="solid")
    #     ws.cell(row=2, column=i+1).font = Font(color="E5F2FC", bold=True)
    #     ws.cell(row=2, column=i+1).alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    # else:
    #     ws.merge_cells(start_row=1, start_column=i+1, end_row=2, end_column=i+1)
    
    # dark blue fill and light blue font
    ws.cell(row=1, column=i+1).fill = PatternFill(start_color="00024A", end_color="00024A", fill_type="solid")
    ws.cell(row=1, column=i+1).font = Font(color="E5F2FC", bold=True)
    ws.cell(row=1, column=i+1).alignment = xl.styles.Alignment(horizontal="center", vertical="center")

'''
# merge B1 C1
ws.merge_cells(start_row=1, start_column=2, end_row=1, end_column=3)

# merge D1 E1
ws.merge_cells(start_row=1, start_column=4, end_row=1, end_column=5)

# merge J1 K1
ws.merge_cells(start_row=1, start_column=10, end_row=1, end_column=11)

# merge M1 N1
ws.merge_cells(start_row=1, start_column=13, end_row=1, end_column=14)

# merge O1 P1
ws.merge_cells(start_row=1, start_column=15, end_row=1, end_column=16)

# merge Q1 R1
ws.merge_cells(start_row=1, start_column=17, end_row=1, end_column=18)

# merge S1 T1
ws.merge_cells(start_row=1, start_column=19, end_row=1, end_column=20)
'''

# endregion

for threshold_iter, thresholds in enumerate(thresholds_to_evaluate):

    clear_output(wait=True)
    
    threshold_protected = thresholds[0]
    threshold_no_protected = thresholds[1]

    protected_ppv = ppv_npv_df[ppv_npv_df["threshold"] == threshold_protected]['protected_ppv'].values[0]
    non_protected_ppv = ppv_npv_df[ppv_npv_df["threshold"] == threshold_no_protected]['unprotected_ppv'].values[0]
    protected_npv = ppv_npv_df[ppv_npv_df["threshold"] == threshold_protected]['protected_npv'].values[0]
    non_protected_npv = ppv_npv_df[ppv_npv_df["threshold"] == threshold_no_protected]['unprotected_npv'].values[0]

    
    directory = f"simulation_outputs/automation_plots/{plot_prefix}_{num_replicas}_iter_plots/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    directory = f"simulation_outputs/automation_plots/{plot_prefix}_{num_replicas}_iter_plots/{threshold_protected}_NPT_{threshold_no_protected}/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    # import patients_with_proba from pkl file
    with open('patients_with_proba.pkl', 'rb') as f:
        patients_og = pickle.load(f) 
        
    all_measures = []
    convergence_values = []

    print(f"Simulating PT {threshold_protected} & NPT {threshold_no_protected} | [P PPV {protected_ppv:.4f}] [NP PPV {non_protected_ppv:.4f}] [P NPV {protected_npv:.4f}] [NP NPV {non_protected_npv:.4f}]")

    # region montecarlo
    convergence = False
    iterations = 0
    while not convergence and iterations < num_replicas:
        start = time.time()
        iterations += 1

        patients=copy.deepcopy(patients_og)
        intermediate=time.time()

        if debug_verbose:
            print(f"After deepcopy {iterations} iterations | {intermediate:.4f}")

        # Initialize appointments (evitar bug en las asignaciones)
        created_appointments = []
        created_appointments = create_appointments(num_serves, simulation_days, work_hours, slot_time)

        # muestreo de los pacientes
        sampled_random_patient_list = random_patient_sample(patients, sample_size, sample_protected_pct, simulation_days)
        
        if debug_verbose:
            print(f"IDs in random_patient_list: {[patient.id for patient in random_patient_list]}")
        
        appointments_from_rule, num_refused = call_a_rule(sampled_random_patient_list, created_appointments, rule_name, model, threshold_protected, threshold_no_protected, overbooking_level=param_overbooking)
        
        if debug_verbose:
            print(f"IDs RandomPatientList after calling rule  : {[patient.id for patient in random_patient_list]}")
            print(f"Appointments before scheduling simulation: {appointments}")
            print(f"\nNumber of patients not assigned {num_refused}")

        for iter in range(iterations_per_schedule):

            # print(f"- Threshold {threshold_iter+1}: Iteration {iter+1}/{iterations_per_schedule}")
            appointments = copy.deepcopy(appointments_from_rule) 
            random_patient_list = copy.deepcopy(sampled_random_patient_list)

            attendance_random_patient_list = stablish_attendance(random_patient_list, 
                                                                 protected_ppv, non_protected_ppv, 
                                                                 protected_npv, non_protected_npv, 
                                                                 threshold_protected, threshold_no_protected)
            
            clinic = Clinic(attendance_random_patient_list, appointments, slot_time)
            clinic.simulation()
            measures = clinic.get_measures()
        
            if debug_verbose:
                print(f"Appointments after scheduling simulation: {appointments}")

                for server in appointments:
                    for i, day in enumerate(server):
                        print(f"\nD{i}")
                        for j, slot in enumerate(day):
                            for appoint_test in slot:
                                print(f"P{appoint_test}|ID{random_patient_list[appoint_test].id}|{random_patient_list[appoint_test].num_slot}", end=", ")
            
            all_measures.append(measures)

            del appointments
            del random_patient_list
        
            if debug_verbose:
                print("measures:", measures)

        if iterations >= 10:
            if iterations%5 == 0:
                convergence, diff = check_convergence_mean(all_measures, verbose=False)
                convergence_values.append(diff)
            else:
                convergence, diff = check_convergence_mean(all_measures)
                convergence_values.append(diff)

        end = time.time()
        print(f"Ran Successfully Replica {iterations}/{num_replicas} in {end-start:.2f}s") # Check iteration print
        
        # Limpiar variables excepto all_measures
        del created_appointments
        del sampled_random_patient_list
        del attendance_random_patient_list
        del clinic
        del patients

    print(f"* * * Ran Successfully All Replicas!!! \n") 
    #endregion

    # region metric_calculations
    measures_df = pd.DataFrame(all_measures)
    summary_measures = get_margin_errors(all_measures)
    summary_measures_df = calculate_summary(measures_df)

    protected_wt = measures_df["protected_overbooked_waiting_time"] / measures_df["protected_overbooked_patients"]
    protected_wt_mean, protected_wt_margin = confidence_interval(protected_wt)

    non_protected_wt = measures_df["non_protected_overbooked_waiting_time"] / measures_df["non_protected_overbooked_patients"]
    non_protected_wt_mean, non_protected_wt_margin = confidence_interval(non_protected_wt)

    idle_time_mean = summary_measures_df['mean'][0]/7
    idle_time_lower_bound = summary_measures_df['confidence_interval'][0][0]/7
    idle_time_upper_bound = summary_measures_df['confidence_interval'][0][1]/7

    over_time_mean = summary_measures_df['mean'][1]/7
    over_time_lower_bound = summary_measures_df['confidence_interval'][1][0]/7
    over_time_upper_bound = summary_measures_df['confidence_interval'][1][1]/7

    no_shows_mean  = summary_measures_df['mean'][2]/7
    no_shows_lower_bound  = summary_measures_df['confidence_interval'][2][0]/7
    no_shows_upper_bound  = summary_measures_df['confidence_interval'][2][1]/7

    patient_wt_mean = summary_measures_df['mean'][13]
    patient_wt_lower_bound = summary_measures_df['confidence_interval'][13][0]
    patient_wt_upper_bound = summary_measures_df['confidence_interval'][13][1]

    ob_prot_wt = measures_df["protected_overbooked_waiting_time"] / measures_df["protected_overbooked_patients"]
    ob_non_prot_wt = measures_df["non_protected_overbooked_waiting_time"] / measures_df["non_protected_overbooked_patients"]

    one_sided_df = ttest(ob_prot_wt, ob_non_prot_wt, correction = False, alternative='greater', confidence=0.95)
    one_sided_df["p-val"].values[0]

    one_sided_correction_df = ttest(ob_prot_wt, ob_non_prot_wt, correction=True, alternative='greater', confidence=0.95)
    one_sided_correction_p_val = one_sided_correction_df["p-val"].values[0]
    one_sided_correction_t = one_sided_correction_df["T"].values[0]

    one_sided_non_correction_df = ttest(ob_prot_wt, ob_non_prot_wt, correction=False, alternative='greater', confidence=0.95)
    one_sided_non_correction_p_val = one_sided_non_correction_df["p-val"].values[0]
    one_sided_non_correction_t = one_sided_non_correction_df["T"].values[0]

    two_sided_correction_df = ttest(ob_prot_wt, ob_non_prot_wt, correction=True, alternative='two-sided', confidence=0.95)
    two_sided_correction_p_val = two_sided_correction_df["p-val"].values[0]
    two_sided_correction_t = two_sided_correction_df["T"].values[0]

    two_sided_non_correction_df = ttest(ob_prot_wt, ob_non_prot_wt, correction=False, alternative='two-sided', confidence=0.95)
    two_sided_non_correction_p_val = two_sided_non_correction_df["p-val"].values[0]
    two_sided_non_correction_t = two_sided_non_correction_df["T"].values[0]
    # endregion

    # region print_metrics
    # PT = Protected threshold | NPT = Non-protected threshold
    if verbose:
        print(f" - - - - {rule_name.replace('_', ' ').title()} | {threshold_protected} PT & {threshold_no_protected} NPT | Metrics - - - - \n")

        print(f"Idle Time per Day: {idle_time_mean:.4f} minutes | ({idle_time_lower_bound:.4f}, {idle_time_upper_bound:.4f}) | Margin {100*((idle_time_upper_bound-idle_time_mean)/idle_time_mean):.4f}%")
        print(f"Over Time per Day: {over_time_mean:.4f} minutes | ({over_time_lower_bound:.4f}, {over_time_upper_bound:.4f}) | Margin {100*((over_time_upper_bound-over_time_mean)/over_time_mean):.4f}%%")
        print(f"No Shows per Day: {no_shows_mean:.4f} patients | ({no_shows_lower_bound:.4f}, {no_shows_upper_bound:.4f}) | Margin {100*((no_shows_upper_bound-no_shows_mean)/no_shows_mean):.4f}%")

        print(f"\nProtected Overbooked Waiting Time: {protected_wt.mean():.4f} minutes | ({protected_wt_mean-protected_wt_margin:.4f}, {protected_wt_mean+protected_wt_margin:.4f}) | Margin {100*(protected_wt_margin/protected_wt_mean):.4f}%")
        print(f"Non-protected Overbooked Waiting Time: {non_protected_wt.mean():.4f} minutes | ({non_protected_wt_mean-non_protected_wt_margin:.4f}, {non_protected_wt_mean+non_protected_wt_margin:.4f}) | Margin {100*(non_protected_wt_margin/non_protected_wt_mean):.4f}%")
        print(f"General patient waiting time: {patient_wt_mean:.4f} minutes | ({patient_wt_lower_bound:.4f}, {patient_wt_upper_bound:.4f}) | Margin {100*((patient_wt_upper_bound-patient_wt_mean)/patient_wt_mean):.4f}%")

        print(f"\nOne-Sided T-Test (Correction) | p-value: {one_sided_correction_p_val:.4f} | t-value: {one_sided_correction_t:.4f}")
        print(f"One-Sided T-Test (Non-Correction) | p-value: {one_sided_non_correction_p_val:.4f} | t-value: {one_sided_non_correction_t:.4f}")
        print(f"Two-Sided T-Test (Correction) | p-value: {two_sided_correction_p_val:.4f} | t-value: {two_sided_correction_t:.4f}")
        print(f"Two-Sided T-Test (Non-Correction) | p-value: {two_sided_non_correction_p_val:.4f} | t-value: {two_sided_non_correction_t:.4f}")
    #endregion

    # region write_metrics to excel
    metric_rows = 5
    starting_row = (threshold_iter*metric_rows)+2

    # A Column: Rule Name
    ws.merge_cells(f"A{starting_row}:A{starting_row+metric_rows-1}")
    ws[f"A{starting_row}"] = f"{rule_name.replace('_', ' ').title()}"
    ws[f"A{starting_row}"].alignment = xl.styles.Alignment(horizontal="left", vertical="center")
    ws[f"A{starting_row}"].border = border_style

    # B Column: Protected sample proportion
    ws.merge_cells(f"B{starting_row}:B{starting_row+metric_rows-1}")
    ws[f"B{starting_row}"] = f"{sample_protected_pct:.4f}"
    ws[f"B{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"B{starting_row}"].border = border_style

    # C Column: Non Protected sample proportion
    ws.merge_cells(f"C{starting_row}:C{starting_row+metric_rows-1}")
    ws[f"C{starting_row}"] = f"{1-sample_protected_pct:.4f}"
    ws[f"C{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"C{starting_row}"].border = border_style

    # D Column: Protected threshold
    ws.merge_cells(f"D{starting_row}:D{starting_row+metric_rows-1}")
    ws[f"D{starting_row}"] = f"{threshold_protected:.4f}"
    ws[f"D{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"D{starting_row}"].border = border_style

    # E Column: Non Protected threshold
    ws.merge_cells(f"E{starting_row}:E{starting_row+metric_rows-1}")
    ws[f"E{starting_row}"] = f"{threshold_no_protected:.4f}"
    ws[f"E{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"E{starting_row}"].border = border_style

    # F Column: Number of iterations
    ws.merge_cells(f"F{starting_row}:F{starting_row+metric_rows-1}")
    ws[f"F{starting_row}"] = f"{iterations}"
    ws[f"F{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"F{starting_row}"].border = border_style

    # G column: Comments
    ws.merge_cells(f"G{starting_row}:G{starting_row+metric_rows-1}")
    ws[f"G{starting_row}"] = f"{(1/3) * sample_protected_pct} interest sample proportion | 1 Fountain Slot"
    ws[f"G{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"G{starting_row}"].border = border_style

    # H column: Metric names 
    metric_names = ["Idle Time", "Over Time", "OB Protected WT", "OB Non Protected WT", "Overall Patient WT"]
    for i, metric_name in enumerate(metric_names):
        ws[f"H{starting_row+i}"] = metric_name
        ws[f"H{starting_row+i}"].alignment = xl.styles.Alignment(horizontal="left", vertical="center")
        ws[f"H{starting_row+i}"].border = border_style
    
    # I column: Metric values
    metric_values = [idle_time_mean, over_time_mean, protected_wt.mean(), non_protected_wt.mean(), patient_wt_mean]
    for i, metric_value in enumerate(metric_values):
        ws[f"I{starting_row+i}"] = metric_value
        ws[f"I{starting_row+i}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
        ws[f"I{starting_row+i}"].border = border_style

    # J column: Metric lower bounds
    metric_lower_bounds = [idle_time_lower_bound, over_time_lower_bound, protected_wt_mean-protected_wt_margin, 
                           non_protected_wt_mean-non_protected_wt_margin, patient_wt_lower_bound]
    for i, metric_lower_bound in enumerate(metric_lower_bounds):
        ws[f"J{starting_row+i}"] = metric_lower_bound
        ws[f"J{starting_row+i}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
        ws[f"J{starting_row+i}"].border = border_style
    
    # K column: Metric upper bounds
    metric_upper_bounds = [idle_time_upper_bound, over_time_upper_bound, protected_wt_mean+protected_wt_margin, 
                           non_protected_wt_mean+non_protected_wt_margin, patient_wt_upper_bound]
    for i, metric_upper_bound in enumerate(metric_upper_bounds):
        ws[f"K{starting_row+i}"] = metric_upper_bound
        ws[f"K{starting_row+i}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
        ws[f"K{starting_row+i}"].border = border_style

    # L column: Margin of interval
    metric_margins = [((idle_time_upper_bound-idle_time_mean)/idle_time_mean), 
                      ((over_time_upper_bound-over_time_mean)/over_time_mean), 
                      (protected_wt_margin/protected_wt_mean), 
                      (non_protected_wt_margin/non_protected_wt_mean), 
                      ((patient_wt_upper_bound-patient_wt_mean)/patient_wt_mean)]
    for i, metric_margin in enumerate(metric_margins):
        ws[f"L{starting_row+i}"] = metric_margin
        ws[f"L{starting_row+i}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
        ws[f"L{starting_row+i}"].border = border_style

    # M column: One-sided P value (No Correction)
    ws.merge_cells(f"M{starting_row}:M{starting_row+metric_rows-1}")
    ws[f"M{starting_row}"] = one_sided_non_correction_p_val
    ws[f"M{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"M{starting_row}"].border = border_style

    # N column: Two-sided P value (No Correction)
    ws.merge_cells(f"N{starting_row}:N{starting_row+metric_rows-1}")
    ws[f"N{starting_row}"] = two_sided_non_correction_p_val
    ws[f"N{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"N{starting_row}"].border = border_style

    # O column: One sided T value (No Correction)
    ws.merge_cells(f"O{starting_row}:O{starting_row+metric_rows-1}")
    ws[f"O{starting_row}"] = one_sided_non_correction_t
    ws[f"O{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"O{starting_row}"].border = border_style

    # P column: Two sided T value (No Correction)
    ws.merge_cells(f"P{starting_row}:P{starting_row+metric_rows-1}")
    ws[f"P{starting_row}"] = two_sided_non_correction_t
    ws[f"P{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"P{starting_row}"].border = border_style

    # Q column: One-sided P value (Correction)
    ws.merge_cells(f"Q{starting_row}:Q{starting_row+metric_rows-1}")
    ws[f"Q{starting_row}"] = one_sided_correction_p_val
    ws[f"Q{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"Q{starting_row}"].border = border_style

    # R column: Two-sided P value (Correction)
    ws.merge_cells(f"R{starting_row}:R{starting_row+metric_rows-1}")
    ws[f"R{starting_row}"] = two_sided_correction_p_val
    ws[f"R{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"R{starting_row}"].border = border_style

    # S column: One sided T value (Correction)
    ws.merge_cells(f"S{starting_row}:S{starting_row+metric_rows-1}")
    ws[f"S{starting_row}"] = one_sided_correction_t
    ws[f"S{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"S{starting_row}"].border = border_style

    # T column: Two sided T value (Correction)
    ws.merge_cells(f"T{starting_row}:T{starting_row+metric_rows-1}")
    ws[f"T{starting_row}"] = two_sided_correction_t
    ws[f"T{starting_row}"].alignment = xl.styles.Alignment(horizontal="center", vertical="center")
    ws[f"T{starting_row}"].border = border_style
    # endregion

    # region generate and write plots in excel
    
    default_image_width = 100
    default_image_height = 80
    default_image_column_width = 13

    # First plot - Overall patient waiting time
    plot_name = "overall_patient_wt_kde"
    overall_waiting_time_data_series = measures_df['patient_waiting_time'].replace([np.inf, -np.inf], np.nan)
    overall_waiting_time_data_series = overall_waiting_time_data_series.dropna()
    plt.figure(figsize=(10,6))
    plt.title('Overall Patient Waiting Time', fontsize=18, fontweight='bold')
    sns.kdeplot(data=overall_waiting_time_data_series, color='midnightblue', linewidth=3, fill=True)
    plt.xlabel('Waiting Time (minutes)', fontsize=18)
    plt.ylabel('Density', fontsize=18)
    plt.grid(alpha=0.5)
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "U"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    # Second plot - Overbooked Waiting Time by Patient boxplot
    plot_name = "wt_patient_class_boxplot"
    plt.figure(figsize=(10,6))
    plt.title('Overbooked Waiting Time by Patient', fontsize=18, fontweight='bold')
    # sns.boxplot(data=[ob_prot_wt, ob_non_prot_wt], color='dodgerblue')

    colors = ['dodgerblue', 'dodgerblue']
    bp = plt.boxplot([ob_prot_wt, ob_non_prot_wt], notch=True, vert=True, patch_artist=True,
            medianprops=dict(color='red'), showfliers=True, widths=0.6)

    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    plt.xlabel('Patient Class', fontsize=18)
    plt.ylabel('Waiting Time (minutes)', fontsize=18)
    # plt.xticks(ticks=[0, 1], labels=['Protected Class', 'Non-Protected Class'], fontsize=14)
    plt.xticks(ticks=[1, 2], labels=['Protected Class', 'Non-Protected Class'], fontsize=14)
    plt.grid(alpha=0.5)
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "V"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    # Overbooked Waiting Time by Patient boxplot
    plot_name = "wt_patient_class_overall_boxplot"
    colors = ['dodgerblue', 'dodgerblue', 'lightgray']
    plt.figure(figsize=(10,6))
    plt.title('Waiting Time by Patient', fontsize=18, fontweight='bold')
    # sns.boxplot(data=[ob_prot_wt, ob_non_prot_wt, measures_df['patient_waiting_time']], palette=colors)

    bp = plt.boxplot([ob_prot_wt, ob_non_prot_wt, measures_df['patient_waiting_time']], notch=True, vert=True, patch_artist=True,
            medianprops=dict(color='red'), showfliers=True, widths=0.6)

    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    plt.xlabel('Patient Class', fontsize=18)
    plt.ylabel('Waiting Time (minutes)', fontsize=18)
    # plt.xticks(ticks=[0, 1, 2], labels=['Overbooked Protected', 'Overbooked Non-Protected', 'Overall'], fontsize=14)
    plt.xticks(ticks=[1, 2, 3], labels=['Overbooked Protected', 'Overbooked Non-Protected', 'Overall'], fontsize=14)
    plt.grid(alpha=0.5)
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")

    column_letter = "W"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    plot_name = "margin_convergence"
    plt.figure(figsize=(10,6))
    plt.title('Convergence through Iterations', fontsize=18, fontweight="bold")
    plt.plot(convergence_values,label='convergence', linewidth=1.5, color='navy')
    plt.xlabel("Iteration", fontsize=18)
    plt.ylabel("Convergence Margin", fontsize=18)
    plt.grid(alpha=0.5)
    # plt.axhline(0.05, color='red', linestyle='--')
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "X"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    # Less useful ones
    plot_name = "idle_time_line_plot"
    idle_time_server_line_plot = plot_line_graph(np.concatenate(measures_df['idle_time_server'].values), 'Idle Time Server', 'Idle Time (minutes)', 'Weekly Server Idle Time')
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "Y"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    plot_name = "over_time_line_plot"
    over_time_server_line_plot = plot_line_graph(np.concatenate(measures_df['over_time'].values), 'Over Time', 'Over Time (minutes)', 'Weekly Server Over Time')
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "Z"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    plot_name = "no_shows_line_plot"
    no_shows_line_plot = plot_line_graph(measures_df['no_shows'].values, 'No Shows', 'No Shows', 'Weekly No-Shows')
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "AA"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    plot_name = "non_protected_wt_line_plot"
    total_wt_non_protected_line_plot = plot_line_graph(measures_df['clients_total_waiting_time non protected class'].values, 'Patients Total Waiting Time', 'Total Waiting Time (minutes)', 'Weekly Non Protected Waiting Time')
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "AB"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    plot_name = "protected_wt_line_plot"
    total_wt_protected_line_plot = plot_line_graph(measures_df['clients_total_waiting_time protected class'].values, 'Patients Total Waiting Time', 'Total Waiting Time (minutes)', 'Weekly Protected Waiting Time')
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "AC"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()

    # Total Waiting Time by Patient Class boxplot
    plot_name = "total_wt_patient_class_boxplot"
    plt.figure(figsize=(10,6))
    plt.title('Total Waiting Time by Patient Class', fontsize=18, fontweight='bold')
    # sns.boxplot(data=[measures_df['clients_total_waiting_time protected class'], measures_df['clients_total_waiting_time non protected class']], color='lightblue')

    colors = ['lightblue', 'lightblue']
    bp = plt.boxplot([measures_df['clients_total_waiting_time protected class'], measures_df['clients_total_waiting_time non protected class']], 
                     notch=True, vert=True, patch_artist=True,
                     medianprops=dict(color='red'), showfliers=True, widths=0.6)

    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    plt.xlabel('Patient Class', fontsize=18)
    plt.ylabel('Total Waiting Time (minutes)', fontsize=18)
    # plt.xticks(ticks=[0, 1], labels=['Protected Class', 'Non-Protected Class'], fontsize=14)
    plt.xticks(ticks=[1, 2], labels=['Protected Class', 'Non-Protected Class'], fontsize=14)
    plt.grid(alpha=0.5)
    plt.savefig(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    column_letter = "AD"
    img = xl.drawing.image.Image(directory + f"{plot_prefix}_PT_{threshold_protected}_NPT_{threshold_no_protected}_{num_replicas}_Iter_{plot_name}.png")
    img.width = default_image_width
    img.height = default_image_height
    ws.merge_cells(f"{column_letter}{starting_row}:{column_letter}{starting_row+metric_rows-1}")
    ws.column_dimensions[f"{column_letter}"].width = default_image_column_width
    img.anchor = f"{column_letter}{starting_row}"
    ws.add_image(img)
    ws[f"{column_letter}{starting_row}"].border = border_style
    plt.show()
    
    #endregion

    wb.save(f"simulation_outputs/excel_files/{plot_prefix}_{num_replicas}_replicas_{iterations_per_schedule}_iter_results.xlsx")

wb.save(f"simulation_outputs/excel_files/{plot_prefix}_{num_replicas}_replicas_{iterations_per_schedule}_iter_final_results.xlsx")

Simulating PT 0.3 & NPT 0.9 | [P PPV 0.1842] [NP PPV 1.0000] [P NPV 0.8888] [NP NPV 0.8614]


Ran Successfully Replica 1/30 in 6.00s


Ran Successfully Replica 2/30 in 5.75s


Ran Successfully Replica 3/30 in 5.72s


Ran Successfully Replica 4/30 in 6.98s


...

Ran Successfully Replica 10/30 in 5.80s


Ran Successfully Replica 11/30 in 5.89s


Ran Successfully Replica 12/30 in 6.30s


Ran Successfully Replica 13/30 in 20.43s


Ran Successfully Replica 14/30 in 13.94s
